In [1]:
import torch

In [2]:
from torchvision import datasets
from torchvision.transforms import ToTensor

train_data = datasets.CIFAR100(
    root="data",
    train=True,
    transform=ToTensor(),
    download=True,
    target_transform=None
    )


test_data = datasets.CIFAR100(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
    target_transform=None
)



Files already downloaded and verified
Files already downloaded and verified
